In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymongo
from pymongo import MongoClient
from keras.utils import Sequence
from keras.utils.np_utils import to_categorical
%load_ext autoreload
%autoreload 2
from tqdm import tqdm_notebook as tqdm


isascii = lambda s: len(s) == len(s.encode())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
IP_ADDRESS = '13.58.253.233'
client = MongoClient(f"mongodb://cjm715:password@{IP_ADDRESS}/py2cpp")
db = client.py2cpp
print(db.python.estimated_document_count(), db.cpp.estimated_document_count())


26114 28064


In [27]:
py_df0 = pd.DataFrame(list(db.python.find()))
cpp_df0 = pd.DataFrame(list(db.cpp.find()))

py_df = py_df0.drop(columns = ['_id'])
cpp_df = cpp_df0.drop(columns = ['_id'])

py_df = py_df.drop_duplicates()
cpp_df = cpp_df.drop_duplicates()

py_df = py_df.reset_index()
cpp_df = cpp_df.reset_index()
py_df = py_df.drop(columns = ['index'])
cpp_df = cpp_df.drop(columns = ['index'])

py_df = py_df[py_df['language'] == 'Python 3']
cpp_df = cpp_df[cpp_df['language'] == 'C++']

py_chall_list = list(py_df['challenge_title'].unique())
cpp_chall_list = list(cpp_df['challenge_title'].unique())
chall_list = list(set(py_chall_list + cpp_chall_list))
py_df = py_df[py_df.challenge_title.isin(chall_list)]
cpp_df = cpp_df[cpp_df.challenge_title.isin(chall_list)]

# Only considers examples with ASCII characters
py_df = py_df[py_df.code.map(lambda s: len(s) == len(s.encode()))]
cpp_df = cpp_df[cpp_df.code.map(lambda s: len(s) == len(s.encode()))]

print(len(py_df),len(cpp_df))

10868 12935


In [31]:
cpp_df['challenge_title'].value_counts().iloc[0:]

Simple Array Sum                      400
Bricks Game                           331
A Very Big Sum                        300
Compare the Triplets                  298
Closest Numbers                       240
Weighted Uniform Strings              240
Funny String                          238
Sherlock and the Valid String         238
Pangrams                              238
The Longest Increasing Subsequence    236
The Love-Letter Mystery               236
Common Child                          232
Highest Value Palindrome              228
The Longest Common Subsequence        227
Bear and Steady Gene                  226
Sherlock and Anagrams                 224
Build a String                        222
String Similarity                     220
Separate the Numbers                  220
Play with words                       216
Find Strings                          208
Coin on the Table                     208
Morgan and a String                   202
Ashton and String                 

In [5]:
idx = 10
print(cpp_df.challenge_title.iloc[idx])
prog_str = cpp_df.code.iloc[idx]
print(prog_str)
with open("prog.cpp", "w") as text_file:
    text_file.write(prog_str)


Simple Array Sum
#include <cmath>
#include <cstdio>
#include <vector>
#include <iostream>
#include <algorithm>
using namespace std;


int main() {
    /* Enter your code here. Read input from STDIN. Print output to STDOUT */ 
    int n,x;
    cin>>n;
    int sum=0;
    for(int i=0;i<n;i++)
    {
       cin>>x;
        sum=sum+x;
    }
    cout<<sum;
    return 0;
}


In [6]:
import subprocess
from subprocess import PIPE
from subprocess import Popen

command = "g++ prog.cpp -o b.out".split()
subprocess.call(command)
with open("input.txt",'rb') as input_file:
    msg = input_file.read()
    proc = Popen("./b.out", stdin=PIPE, stdout=PIPE)
    out,err = proc.communicate(input=msg)
    out = float(out)
    print(out==31)
    print(err)
    

True
None


In [7]:
#df = pd.concat([py_df,cpp_df])


In [29]:
from tqdm import tqdm_notebook as tqdm

train_df = pd.DataFrame()
challenge_title_list = []
python_code_list = []
cpp_code_list = []

for _, py_ex in tqdm(py_df.iterrows()):

    num_per_ex = 18
    temp_df = cpp_df[cpp_df['challenge_title']==py_ex.challenge_title]
    if len(temp_df)< num_per_ex:
        continue
        
    temp_df = temp_df.sample(n = num_per_ex)
    
    for i in range(num_per_ex):
        cpp_ex_code = temp_df['code'].iloc[i]

        challenge_title_list.append(py_ex.challenge_title)
        python_code_list.append(py_ex.code)
        cpp_code_list.append(cpp_ex_code)
    
        
df_as_dict = {'challenge_title' : challenge_title_list,
           'python_code' : python_code_list,
             'cpp_code' : cpp_code_list}    

train_df = pd.DataFrame.from_dict(df_as_dict)
train_df = train_df.drop_duplicates()
train_df

,challenge_title,python_code,cpp_code
0,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
1,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
2,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
3,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
4,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
5,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
6,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
7,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
8,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
9,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...


In [30]:
train_df.to_pickle('train_df.pkl')

In [10]:
train_df['challenge_title'].value_counts()

Simple Array Sum                      7704
Compare the Triplets                  6030
A Very Big Sum                        5130
Weighted Uniform Strings              5116
Pangrams                              4284
The Longest Increasing Subsequence    4122
Sherlock and the Valid String         3875
The Love-Letter Mystery               3868
Closest Numbers                       3849
Funny String                          3838
Bear and Steady Gene                  3759
Separate the Numbers                  3758
Highest Value Palindrome              3724
Morgan and a String                   3594
Sherlock and Anagrams                 3478
Diagonal Difference                   3366
The Longest Common Subsequence        3361
Apple and Orange                      2574
Between Two Sets                      2484
Breaking the Records                  2466
Plus Minus                            2466
Time Conversion                       2412
Common Child                          2274
String Simi

In [23]:
len(py_df[py_df.code.map(lambda s: len(s) == len(s.encode()))])

10788

In [17]:
# a = '¥'
# a = 'c'

In [18]:
isascii(a)

True

,challenge_title,python_code,cpp_code
77424,Funny String,t = int(input())\nfor tt in range(t):\n s =...,#include <cmath>\n#include <cstdio>\n#include ...
66796,Bricks Game,for case_t in range(int(input())):\n answer...,#include <cmath>\n#include <cstdio>\n#include ...
77437,Funny String,t = int(input())\n\nfor _ in range(t):\n s ...,#include <cmath>\n#include <cstdio>\n#include ...
62878,Weighted Uniform Strings,#!/bin/python3\n\nimport sys\n\n\ndef value_c(...,#include <bits/stdc++.h>\nusing namespace std;...
125899,Gridland Provinces,"# -*- coding: utf-8 -*-\n""""""\nCreated on Tue J...",#include <bits/stdc++.h>\nusing namespace std;...
176670,Even Tree,from collections import defaultdict\n\n\ndef _...,/* Enter your code here. Read input from STDIN...
62285,Weighted Uniform Strings,#!/bin/python3\n\nimport sys\n\n\nstr1 = input...,#include <map>\n#include <set>\n#include <list...
36764,Append and Delete,#!/bin/python3\ns = input().strip()\nt = input...,#include <algorithm>\n#include <bitset>\n#incl...
145311,Closest Numbers,"def abs(a,b):\n if a>b:\n return a-b...",#include <iostream>\n#include <string>\n#inclu...
80801,The Longest Common Subsequence,"n, m = map(int, input().split())\nx = list(map...",#include <cmath>\n#include <cstdio>\n#include ...
